# Part I - reading postal codes (scroll down for tasks 2 and 3)


Import all necessary packages

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Download the page into parser

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup (r.text, "html.parser")

Parse page into a dictionary where each postal code is represented with one line

In [3]:
NA  ="Not assigned" # A constant for "Not assigned marker"

pc_dict = dict() # A dictionary for postal codes. One postal code - one entry in the dict.

rows = soup.table.find_all("tr") # The data are in the first table. Let's read line by line
for r in rows:
    tds = r.find_all("td")
    if len(tds) > 0:                     # ignore headings
        code = tds[0].text.strip()       # remove extra characters (there are some!)
        borough = tds[1].text.strip()
        neigh = tds[2].text.strip()
        
        if (borough != NA):              # Ignore lines where borough is unassigned
            if (neigh == NA):            # Fill in unassigned neighbourhood
                neigh = borough
            if code in pc_dict:          # Did we see this postal code before?
                line = pc_dict[code]     # ... yes - then just add another neighbourhood
                line[2] += "," + neigh
            else:                        # ... no - create a new entry
                pc_dict[code] = [code, borough, neigh]
#print(pc_dict)

Transform dictionary into dataframe

In [4]:
postalCodes = pd.DataFrame(list(pc_dict.values()), columns = ['PostalCode', 'Borough', 'Neighborhood'])
postalCodes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [5]:
postalCodes.shape

(103, 3)

# Part 2 - Get coordinates

Time to get coordinates for each postal code.

First, let's try geocoder as in the assignment description

In [6]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
inf_loop_braker = 0
while(lat_lng_coords is None and inf_loop_braker < 5):
    g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
    lat_lng_coords = g.latlng
    print ("Iteration: {}, coordinates: {}".format(inf_loop_braker, lat_lng_coords))
    inf_loop_braker += 1


Iteration: 0, coordinates: None
Iteration: 1, coordinates: None
Iteration: 2, coordinates: None
Iteration: 3, coordinates: None
Iteration: 4, coordinates: None


No matter gow many time I try and what address I use it is always None

Then, let's try Nominatim as in the lab

In [7]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ny_explorer")
def test_address(address):
    location = geolocator.geocode(address)
    if location is None:
        print ("Location '{}' not found".format(address))
    else:
        print ("Coordinates for '{}': {}, {}".format(address, location.latitude, location.longitude))

In [8]:
test_address("M5A, Toronto, Ontario")
test_address("M5A, Toronto, Ontario, Canada")
test_address("M5G, Toronto, Ontario")
test_address("M5G, Toronto, Ontario, Canada")

Coordinates for 'M5A, Toronto, Ontario': 43.6636637, -79.3675307
Coordinates for 'M5A, Toronto, Ontario, Canada': 43.6636637, -79.3675307
Location 'M5G, Toronto, Ontario' not found
Location 'M5G, Toronto, Ontario, Canada' not found


In [9]:
n_filled = 0
for i in postalCodes.index:
    address = "{}, Toronto, Ontario".format(postalCodes.PostalCode[i])
    location = geolocator.geocode(address)
    if not location is None:
        n_filled += 1
        
print ("Number of found zip codes: {}".format(n_filled))

Number of found zip codes: 23


Some zip codes are recogined, but only minor part. And even for those recognized coordinates are different than in "Geospacial_coordinates.csv".

Thus for consistency let's use Geospacial_coordinates.csv

In [10]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
postalCodes = postalCodes.merge(coordinates, left_on="PostalCode", right_on="Postal Code").drop(columns=["Postal Code"])
postalCodes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# Part 3, explore, visualize and cluster

### Let's start with simply showing all the neighborhoods on the map

In [12]:
import folium

# Create a map centered at Toronto
location = geolocator.geocode("Toronto, Canada")
map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# Select only boroughs with "Toronto" in the name
torontoCodes = postalCodes[postalCodes.Borough.apply(lambda x: "Toronto" in x)]

# Add markers for each Neighborhood (or more precisely - for each postal code)
for lat, lng, borough, neighborhood in zip(torontoCodes['Latitude'], torontoCodes['Longitude'], torontoCodes['Borough'], torontoCodes['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Venues data

Now let get venues from FourSquare. **In order to run further cells you need to  type in your credentials in the below box or create a file with Foursquare ID and secret.**

In [13]:
CLIENT_ID = 'REPLACE WITH YOUR ID OR CREATE A FILE FS_cred.txt' # your Foursquare ID
CLIENT_SECRET = 'REPLACE WITH YOUR SECRET OR CREATE A FILE FS_cred.txt' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


try:
    filename = "FS_cred.txt" 
    # This block will read 
    with open(filename, "r") as f:
        lines = f.readlines()
        CLIENT_ID = lines[0].strip()
        CLIENT_SECRET = lines[1].strip()
        print ("Foursquare credentials read from '{}'".format(filename))
except:
    print ("Could not read '{}'".format(filename))

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)


Foursquare credentials read from 'FS_cred.txt'


Get venues around "zipcode center" location

In [14]:
LIMIT = 100
def getNearbyVenues(codes, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for code, name, lat, lng in zip(codes, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            code,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zipcode', 
                  'Neighborhood',           
                  'Zipcode Latitude', 
                  'Zipcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(  codes=torontoCodes['PostalCode'],
                                   names=torontoCodes['Neighborhood'],
                                   latitudes=torontoCodes['Latitude'],
                                   longitudes=torontoCodes['Longitude']
                                  )


Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(1698, 8)


,Zipcode,Neighborhood,Zipcode Latitude,Zipcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,M5A,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,M5A,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In most cases number of venues is below the limit of 100. Several neighborhood have less than 10 venues

In [17]:
toronto_venue_counts = toronto_venues.groupby('Neighborhood').count().reset_index()[["Neighborhood", "Venue"]]
toronto_venue_counts.columns = ["Neighborhood", "N of Venues"]
toronto_venue_counts

,Neighborhood,N of Venues
0,"Adelaide,King,Richmond",100
1,Berczy Park,55
2,"Brockton,Exhibition Place,Parkdale Village",20
3,Business Reply Mail Processing Centre 969 Eastern,17
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",14
5,"Cabbagetown,St. James Town",45
6,Central Bay Street,82
7,"Chinatown,Grange Park,Kensington Market",100
8,Christie,15
9,Church and Wellesley,88


In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


In the following cells neighborhood will be clustered by ratio of various venue categories. 

Convert categorical variable into one-hot encoding and then to ratio of each category within a neighborhood:

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.01
1,Berczy Park,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00


Create a dataframe that shows 10 most common categories:

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Clothing Store,Asian Restaurant,Restaurant,Gym,Hotel
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market,Italian Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Breakfast Spot,Café,Performing Arts Venue,Burrito Place,Stadium,Bar,Caribbean Restaurant,Italian Restaurant,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop,Farmers Market
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Airport Service,Plane,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Harbor / Marina


Perform clustering

In [21]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = torontoCodes

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.join(toronto_venue_counts.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,N of Venues
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Pub,Park,Theater,Mexican Restaurant,Restaurant,Breakfast Spot,Electronics Store,49
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Tea Room,Japanese Restaurant,Italian Restaurant,Pizza Place,Diner,100
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Hotel,Café,Clothing Store,Italian Restaurant,Park,Cosmetics Shop,Cocktail Bar,Breakfast Spot,100
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Coffee Shop,Pub,Women's Store,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,4
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market,Italian Restaurant,55


Visualize clustering

In [23]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
location = geolocator.geocode("Toronto, Canada")
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

As we see on the map, clustering did not produce any significant grouping. Most of points went to cluster 0. Other clusters are formed from the neighborhoods witrh only 1-4 venues. Changing target number of clusters does not help.

A hypothesis that could be tested as a next step: 
* group categories into some super-categories, such as drink/food, shops, parks, other
* cluster by number of national cuisine restaurants
* number or venues may be one best single criteria for clustering

Choice of hypothesis to test next would depend on task context (if it was real world task)

In [24]:
toronto_merged.groupby("Cluster Labels").count().reset_index()[["Cluster Labels", "PostalCode"]]

,Cluster Labels,PostalCode
0,0,32
1,1,1
2,2,1
3,3,3
4,4,1


In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] != 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,N of Venues
19,East Toronto,4,Coffee Shop,Pub,Women's Store,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,4
61,Central Toronto,3,Bus Line,Park,Swim School,Dim Sum Restaurant,Women's Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,4
62,Central Toronto,1,Garden,Women's Store,Fish Market,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,1
68,Central Toronto,3,Trail,Jewelry Store,Sushi Restaurant,Park,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Women's Store,4
83,Central Toronto,2,Restaurant,Trail,Summer Camp,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,3
91,Downtown Toronto,3,Park,Trail,Playground,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Women's Store,4
